In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import make_scorer
from sklearn.svm import LinearSVR
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
%matplotlib qt

In [3]:
from random import random, randrange, sample


def mix_augument(np_x, np_y, item_num = 10, drop_out = 0):
    aug_x = np.zeros((item_num, np_x.shape[1]))
    aug_y = np.zeros(item_num)
    for i in range(item_num):
        aug = sample(range(np_x.shape[0]), k=2)
        mask_a = np.ones(np_x.shape[1])
        mask_b = np.ones(np_x.shape[1])
        num_drop = int(np_x.shape[1] * drop_out)
        mask_a[:num_drop] = 0
        mask_b[:num_drop] = 0
        np.random.shuffle(mask_a)
        np.random.shuffle(mask_b)
        a = aug[0]
        b = aug[1]
        #print(f'{a}{b}')
        wa = random()
        wb = 1-wa
        #print(b)
        np_a = np_x[a,:] 
        np_b = np_x[b,:] 
        aug_x[i] = wa*np_a*mask_a + wb*np_b*mask_b
        aug_y[i] = wa*np_y[a] + wb*np_y[b]
        # print(mask_a)
        # plt.figure(1)
        # plt.plot(np_mass_features_ms5, np_a)
        # plt.ylim((0,1))
        # plt.show()
        # plt.figure(2)
        # plt.ylim((0,1))
        # plt.plot(np_mass_features_ms5, np_a*mask_a)
        # plt.show()
        # plt.figure(3)
        # plt.plot(np_mass_features_ms5, np_b)
        # plt.ylim((0,1))
        # plt.show()
        # plt.figure(4)
        # plt.ylim((0,1))
        # plt.plot(np_mass_features_ms5, np_b*mask_b)
        # plt.show()
        # plt.figure()
        # plt.ylim((0,1))
        # plt.plot(np_mass_features_ms5, aug_x[i] / max(aug_x[i]))
        # plt.show()
    aug_x = np.concatenate((np_x, aug_x))
    aug_y = np.concatenate((np_y, aug_y))
    return aug_x, aug_y



In [4]:
datasets_NIST = ['G2S1(6)', 'G2S2(6)', 'G2FS1(6)', 'G2FS2(6)','G2FS2(6)_REDO','G2S2(3)', 'G2FS2(3)']
datasets_NIST_train = ['G2S1(6)', 'G2S2(6)', 'G2FS1(6)', 'G2FS2(6)_REDO','G2S2(3)', 'G2FS2(3)']
datasets_NIST_old = ['G2S1(6)', 'G2S2(6)', 'G2FS1(6)', 'G2FS2(6)','G2S2(3)', 'G2FS2(3)']
datasets_SL = ['SL']
datasets_Released = ['ControlFetuin', 'ControlTransferrin','Alpha.2,3Neur.Fetuin']
def load_datasets(datasets, option = '', level = 'MS5'):
    if option:
        option = option + '_'
    if level == 'MS5':
        resolution = 10000
    else:
        resolution = 23000
    np_input = np.empty(shape=[resolution,0])
    np_output = np.empty(shape=[0])
    df_metadata = pd.DataFrame()
    for currd in datasets:
        path = f'data\\test_{option}{currd}_{level}'
        # NOTE: Make sure that the outcome column is labeled 'target' in the data file
        try: 
            df_input = pd.read_csv(path + '\\' + 'input.csv', header = None)
            df_output = pd.read_csv(path + '\\' +'output.csv', header = None)
            df_currmeta = pd.read_csv(path+'\\'+'metadata.csv', header=None, skiprows=1)
        except Exception as E:
            print(f'Error loading {currd}')
            print(E)
            continue
        df_currmeta[4] = currd
        if currd in datasets_NIST:
            df_currmeta[5] = 'NIST'
        elif currd in datasets_Released:
            df_currmeta[5] = 'UGA-R'
        else:
            df_currmeta[5] = 'UGA-S'
        df_metadata = pd.concat((df_metadata, df_currmeta))
        df_mass_features_ms5 = pd.read_csv(path + '\\' +'mass_features.csv', header = None)
        np_mass_features_ms5 = df_mass_features_ms5.to_numpy()
        np_mass_features_ms5 = np_mass_features_ms5.reshape(-1)
        np_input_curr = df_input.to_numpy()
        np_output_curr = df_output.to_numpy()
        np_output_curr = np_output_curr.T
        np_output_curr = np_output_curr.reshape(-1)
        np_input = np.append(np_input, np_input_curr, axis=1)
        np_output = np.append(np_output, np_output_curr)
    return np_input, np_output, np_mass_features_ms5,df_metadata

In [5]:
def mix_augument_2(np_x, np_y, item_num = 10, drop_out = 0):
    aug_x = None
    aug_y = None
    item_num_ratio = int(item_num / 5)
    ratio_list = [0, 0.1, 0.5, 0.9, 1]
    for ratio in ratio_list:
        idx = np_y == ratio
        ratio_x = np_x[idx, :]
        ratio_y = np_y[idx]
        aug_ratio_x, aug_ratio_y = mix_augument(ratio_x, ratio_y, item_num= item_num_ratio, drop_out=drop_out)
        aug_ratio_y = np.ones(len(aug_ratio_y)) * ratio
        if aug_x is None:
            aug_x = aug_ratio_x
        else:
            aug_x = np.concatenate((aug_x, aug_ratio_x))
        if aug_y is None:
            aug_y = aug_ratio_y
        else:
            aug_y = np.concatenate((aug_y, aug_ratio_y))

    return aug_x, aug_y



In [6]:
def resample_augument(np_x, np_y, item_num = 10, drop_out = 0):
    aug = sample(range(np_x.shape[0]), k=item_num)
    aug_x = np_x[aug, :]
    aug_y = np_y[aug]
    aug_x = np.concatenate((np_x, aug_x))
    aug_y = np.concatenate((np_y, aug_y))
    return aug_x, aug_y

In [7]:
def group_by_ratio(np_x, np_y, **args):
    aug_x = None
    aug_y = None
    ratio_list = [0, 0.1, 0.5, 0.9, 1, -1]
    for ratio in ratio_list:
        idx = np_y == ratio
        ratio_x = np_x[idx, :]
        ratio_y = np_y[idx]
        if aug_x is None:
            aug_x = ratio_x
        else:
            aug_x = np.concatenate((aug_x, ratio_x))
        if aug_y is None:
            aug_y = ratio_y
        else:
            aug_y = np.concatenate((aug_y, ratio_y))

    return aug_x, aug_y



In [8]:
def customize_score(true_value, predict):
    #predict = 1/(1+np.exp(-predict))
    predict = np.clip(predict,0,1)
    return -mean_squared_error(true_value, predict, squared=False)
my_scorer = make_scorer(customize_score, greater_is_better=True)

In [9]:
class Normalized_NMF(NMF):
    def __init__(
        self,
        n_components=None,
        *,
        init="warn",
        solver="cd",
        beta_loss="frobenius",
        tol=1e-4,
        max_iter=200,
        random_state=None,
        alpha="deprecated",
        alpha_W=0.0,
        alpha_H="same",
        l1_ratio=0.0,
        verbose=0,
        shuffle=False,
        regularization="deprecated",
        normalized = 0,
    ):
        self.n_components = n_components
        self.init = init
        self.solver = solver
        self.beta_loss = beta_loss
        self.tol = tol
        self.max_iter = max_iter
        self.random_state = random_state
        self.alpha = alpha
        self.alpha_W = alpha_W
        self.alpha_H = alpha_H
        self.l1_ratio = l1_ratio
        self.verbose = verbose
        self.shuffle = shuffle
        self.regularization = regularization
        self.normalized = normalized


    
    def transform(self, X):
        W = super().transform(X) # n x l
        X_est = super().inverse_transform(W) # n x m
        if self.normalized > 0:
            X_max = np.max(X_est, axis = 1) # n x 1
            X_max[X_max < self.normalized] = 1
            W = np.nan_to_num(W / X_max[:, np.newaxis])
        return W


In [10]:
preprocessing_pipeline = Pipeline([
    ('logT', FunctionTransformer(np.log1p)),
    ('Norm', Normalizer()),
])

outlier_pipeline = Pipeline([
    # ('logT', FunctionTransformer(np.log1p)),
    # ('Norm', Normalizer()),
    ('NMF',NMF(2, random_state=42, max_iter=100000, init = 'nndsvd'))
])

regressor_pipeline = Pipeline([
    ('Standardlize', StandardScaler(with_mean=True)),
    #('PCA', PCA()),
    #('var', VarianceThreshold(threshold=0.001)),
    #('select', SelectPercentile(score_func=f_regression, percentile = 80)),
    #('pca', PCA()),
    ('LR', LinearSVR(random_state = 42, fit_intercept=True, max_iter=100000))]
)


# exported_pipeline = make_pipeline(
# #     StackingEstimator(estimator=DecisionTreeRegressor(max_depth=7, min_samples_leaf=4, min_samples_split=18)),
#     Normalizer(norm="max"),
#     VarianceThreshold(threshold=0.005),
# #     RobustScaler(),
#     SelectPercentile(score_func=f_regression, percentile=37),
#     LinearSVR(C=1.0, dual=True, epsilon=0.0001, loss="epsilon_insensitive", tol=0.0001, max_iter = 10000)
# )

#exported_pipeline.fit(np_input, np_output)
#results = exported_pipeline.predict(np_input)
#print(results)

In [11]:
param_grid = {
    # "var__threshold": np.linspace(0.0001, 0.01, 5),
    # "select__percentile": np.linspace(10, 100, 5),
    #'pca__n_components': [5, 10, 15, 20, 25],
    # 'model__eta': [0.1,0.2,0.3],
    # 'model__max_depth': [2,3],
    #'model__gamma': [0, 1, 2],
    #'model__n_estimators': [500],
    #'model__reg_alpha':[ 0,0.1,0.2,0.3],
    #'model__reg_lambda':[0,0.1,0.2,0.3]
    # 'model__colsample_bytree': [0.3,0.7,1],
    
#     "variancethreshold__threshold": np.linspace(0.0001, 0.01, 20),
#     "selectpercentile__percentile": np.linspace(10, 100, 10),
    # "PCA__n_components": [2,3,4,8,16],
    # "NMF__n_components": [2,4,8],
    #"NMF__normalized": [0, 0.8],
    "LR__fit_intercept": [True, False],
    "Standardlize__with_mean": [True, False],
 #   "NMF__alpha_H": [0, 0.001],
 #   "NMF__alpha_W": [0, 0.001],
 #   "LR__C": [0.1,0.5,1],
 #   "LR__epsilon": [0.001,0.01,0.1,0],
}

## Mask non-theoretical peaks
* a2-3 signature peaks: 103.08, 131.07
* a2-6 signature peaks: 89.06, 117.06

# Experiments

In [12]:
train_threshold = 0.8
test_threshold = 0.8

In [34]:
# Inter-dataset prediction
SL_NIST_rmse = []
print('SL -> NIST')
test_x, test_y, _,test_meta = load_datasets(datasets_NIST, 'NMF_template')
train_x, train_y, mass_features,train_meta = load_datasets(datasets_SL)
p_x = train_x
scalery = StandardScaler()
train_x = preprocessing_pipeline.fit_transform(train_x.T).T
test_x = preprocessing_pipeline.transform(test_x.T).T
# outlier detection for training set
outlier_pipeline.fit(np.concatenate((train_x.T, test_x.T)))
W_x = outlier_pipeline.transform(train_x.T)
est_x = outlier_pipeline.inverse_transform(W_x)
cs_x = cosine_similarity(est_x, train_x.T)
weight_x = cs_x.diagonal()
idx = weight_x > train_threshold
SL_NIST_train_idx = idx
train_x = W_x[idx,:]
train_y = train_y[idx]
train_meta = train_meta.iloc[idx]
# outlier detection for test set
W_x = outlier_pipeline.transform(test_x.T)
est_x = outlier_pipeline.inverse_transform(W_x)
cs_x = cosine_similarity(est_x, test_x.T)
weight_x = cs_x.diagonal()
clip = weight_x > test_threshold
SL_NIST_test_idx = clip
test_x = W_x[clip,:]
test_y = test_y[clip]
test_meta = test_meta.iloc[clip]
model = regressor_pipeline.fit(train_x, scalery.fit_transform(train_y.reshape((-1,1))).reshape(-1))
print(f'training set samples:{train_x.shape[0]} test set samples:{test_x.shape[0]}')
from collections import defaultdict
pred_dict = defaultdict(lambda : [])
gt_dict = defaultdict(lambda : [])
pred = np.clip(scalery.inverse_transform(model.predict(test_x).reshape((-1,1))), 0, 1)
for i in range(pred.shape[0]):
    key = test_meta.iloc[i][0] + str(test_meta.iloc[i][2])
    pred_dict[key].append(pred[i])
    gt_dict[key].append(test_y[i])
isolation_window = ['A', 'B', 'C']
energy_level = [30 , 35, 40, 45, 50]
all_pred = []
all_gt = []
for (i, iw) in enumerate(isolation_window):
    for (j, el) in enumerate(energy_level):
        key = iw + str(el)
        # print(key)
        rmse = mean_squared_error(pred_dict[key], gt_dict[key], squared=False)
        # f1 = f1_score(pred_dict[key], gt_dict[key])
        print(f'{iw}{el} | rmse: {"%.3f" % rmse}')
        SL_NIST_rmse.append(rmse)
        all_pred = all_pred + pred_dict[key]
        all_gt = all_gt + gt_dict[key]

rmse = mean_squared_error(all_pred, all_gt, squared = False)
# f1 = f1_score(all_pred, all_gt)
print(f'all | rmse: {"%.3f" % rmse}')
SL_NIST_rmse.append(rmse)
for gt in [0, 0.1, 0.5, 0.9, 1]:
    idx = np.array(all_gt) == gt
    if idx.any():
        print(f'ground truth: {gt} | prediction variance: {"%.3f" % np.var(np.array(all_pred)[idx])}')

SL -> NIST
training set samples:78 test set samples:73
A30 | rmse: 0.043
A35 | rmse: 0.104
A40 | rmse: 0.053
A45 | rmse: 0.040
A50 | rmse: 0.088
B30 | rmse: 0.060
B35 | rmse: 0.130
B40 | rmse: 0.019
B45 | rmse: 0.002
B50 | rmse: 0.055
C30 | rmse: 0.092
C35 | rmse: 0.111
C40 | rmse: 0.012
C45 | rmse: 0.061
C50 | rmse: 0.034
all | rmse: 0.072
ground truth: 0 | prediction variance: 0.005
ground truth: 1 | prediction variance: 0.000


In [18]:
# Inter-dataset prediction
print('NIST -> SL')
test_x, test_y, _,test_meta = load_datasets(datasets_SL, 'NMF_template')
train_x, train_y, _,train_meta = load_datasets(datasets_NIST_train)
scalery = StandardScaler()
train_x = preprocessing_pipeline.fit_transform(train_x.T).T
test_x = preprocessing_pipeline.transform(test_x.T).T
# outlier detection for training set
outlier_pipeline.fit(np.concatenate((train_x.T, test_x.T)))
W_x = outlier_pipeline.transform(train_x.T)
est_x = outlier_pipeline.inverse_transform(W_x)
cs_x = cosine_similarity(est_x, train_x.T)
weight_x = cs_x.diagonal()
idx = weight_x > train_threshold
train_x = W_x[idx,:]
train_y = train_y[idx]
train_meta = train_meta.iloc[idx]
# outlier detection for test set
W_x = outlier_pipeline.transform(test_x.T)
est_x = outlier_pipeline.inverse_transform(W_x)
cs_x = cosine_similarity(est_x, test_x.T)
weight_x = cs_x.diagonal()
clip = weight_x > test_threshold
test_x = W_x[clip,:]
test_y = test_y[clip]
test_meta = test_meta.iloc[clip]
model = regressor_pipeline.fit(train_x, scalery.fit_transform(train_y.reshape((-1,1))).reshape(-1))
print(f'training set samples:{train_x.shape[0]} test set samples:{test_x.shape[0]}')
from collections import defaultdict
pred_dict = defaultdict(lambda : [])
gt_dict = defaultdict(lambda : [])
pred = np.clip(scalery.inverse_transform(model.predict(test_x).reshape((-1,1))), 0, 1)
for i in range(pred.shape[0]):
    key = test_meta.iloc[i][0] + str(test_meta.iloc[i][2])
    pred_dict[key].append(pred[i])
    gt_dict[key].append(test_y[i])
isolation_window = ['A', 'B', 'C']
energy_level = [30 , 35, 40, 45, 50]
all_pred = []
all_gt = []
for (i, iw) in enumerate(isolation_window):
    for (j, el) in enumerate(energy_level):
        key = iw + str(el)
        # print(key)
        rmse = mean_squared_error(pred_dict[key], gt_dict[key], squared=False)
        # f1 = f1_score(pred_dict[key], gt_dict[key])
        print(f'{iw}{el} | rmse: {"%.3f" % rmse}')
        all_pred = all_pred + pred_dict[key]
        all_gt = all_gt + gt_dict[key]

rmse = mean_squared_error(all_pred, all_gt, squared = False)
# f1 = f1_score(all_pred, all_gt)
print(f'all | rmse: {"%.3f" % rmse}')
for gt in [0, 0.1, 0.5, 0.9, 1]:
    idx = np.array(all_gt) == gt
    if idx.any():
        print(f'ground truth: {gt} | prediction std: {"%.3f" % np.std(np.array(all_pred)[idx])}')

NIST -> SL
training set samples:81 test set samples:78
A30 | rmse: 0.074
A35 | rmse: 0.060
A40 | rmse: 0.064
A45 | rmse: 0.050
A50 | rmse: 0.058
B30 | rmse: 0.077
B35 | rmse: 0.057
B40 | rmse: 0.053
B45 | rmse: 0.054
B50 | rmse: 0.053
C30 | rmse: 0.081
C35 | rmse: 0.058
C40 | rmse: 0.056
C45 | rmse: 0.058
C50 | rmse: 0.054
all | rmse: 0.061
ground truth: 0 | prediction std: 0.006
ground truth: 0.1 | prediction std: 0.012
ground truth: 0.5 | prediction std: 0.021
ground truth: 0.9 | prediction std: 0.006
ground truth: 1 | prediction std: 0.002


In [19]:
# Intra-dataset prediction
print('SL -> SL')
from collections import defaultdict
from sklearn.model_selection import KFold
np_x, np_y,_, df_meta = load_datasets(datasets_SL)
idx = np.arange(np_y.shape[0])
np.random.shuffle(idx)
np_x = np_x[:, idx]
np_y = np_y[idx]
df_meta = df_meta.iloc[idx]
outlier_pipeline.fit(np_x.T)
pred_dict = defaultdict(lambda : [])
gt_dict = defaultdict(lambda : [])
k_fold = KFold(n_splits=5)
for trainidx, testidx in k_fold.split(np_x.T):
    train_x = np_x[:, trainidx]
    train_y = np_y[trainidx]
    test_x = np_x[:, testidx]
    test_y = np_y[testidx]
    df_test = df_meta.iloc[testidx]
    train_x = preprocessing_pipeline.fit_transform(train_x.T).T
    test_x = preprocessing_pipeline.transform(test_x.T).T
    # outlier detection for training set
    W_x = outlier_pipeline.transform(train_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, train_x.T)
    weight_x = cs_x.diagonal()
    idx = weight_x > train_threshold
    train_x = W_x[idx,:]
    train_y = train_y[idx]
    # outlier detection for test set
    W_x = outlier_pipeline.transform(test_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, test_x.T)
    weight_x = cs_x.diagonal()
    clip = weight_x > test_threshold
    test_x = W_x[clip,:]
    test_y = test_y[clip]
    if ~clip.all():
        print(f'{df_test[~clip]} is removed')
    df_test = df_test.iloc[clip]
    regressor_pipeline.fit(train_x, scalery.fit_transform(train_y.reshape((-1,1))).reshape(-1))
    pred = np.clip(scalery.inverse_transform(regressor_pipeline.predict(test_x).reshape((-1,1))), 0, 1)
    for i in range(pred.shape[0]):
        key = df_test.iloc[i][0] + str(df_test.iloc[i][2])
        pred_dict[key].append(pred[i])
        gt_dict[key].append(test_y[i])
isolation_window = ['A', 'B', 'C']
energy_level = [30 , 35, 40, 45, 50]
all_pred = []
all_gt = []
for (i, iw) in enumerate(isolation_window):
    for (j, el) in enumerate(energy_level):
        key = iw + str(el)
        rmse = mean_squared_error(pred_dict[key], gt_dict[key], squared=False)
        # f1 = f1_score(pred_dict[key], gt_dict[key])
        print(f'{iw}{el} | rmse: {"%.3f" % rmse}')
        all_pred = all_pred + pred_dict[key]
        all_gt = all_gt + gt_dict[key]

rmse = mean_squared_error(all_pred, all_gt, squared = False)
# f1 = f1_score(all_pred, all_gt)
print(f'all | rmse: {"%.3f" % rmse}')

SL -> SL
A30 | rmse: 0.028
A35 | rmse: 0.016
A40 | rmse: 0.017
A45 | rmse: 0.008
A50 | rmse: 0.014
B30 | rmse: 0.037
B35 | rmse: 0.021
B40 | rmse: 0.014
B45 | rmse: 0.011
B50 | rmse: 0.008
C30 | rmse: 0.036
C35 | rmse: 0.018
C40 | rmse: 0.011
C45 | rmse: 0.007
C50 | rmse: 0.009
all | rmse: 0.019


In [20]:
# Intra-dataset prediction
print('NIST -> NIST')
from collections import defaultdict
from sklearn.model_selection import KFold
np_x, np_y,_, df_meta = load_datasets(datasets_NIST_train)
idx = np.arange(np_y.shape[0])
np.random.shuffle(idx)
np_x = np_x[:, idx]
np_y = np_y[idx]
df_meta = df_meta.iloc[idx]
outlier_pipeline.fit(np_x.T)
pred_dict = defaultdict(lambda : [])
gt_dict = defaultdict(lambda : [])
k_fold = KFold(n_splits=5)
for trainidx, testidx in k_fold.split(np_x.T):
    train_x = np_x[:, trainidx]
    train_y = np_y[trainidx]
    test_x = np_x[:, testidx]
    test_y = np_y[testidx]
    df_test = df_meta.iloc[testidx]
    train_x = preprocessing_pipeline.fit_transform(train_x.T).T
    test_x = preprocessing_pipeline.transform(test_x.T).T
    # outlier detection for training set
    # outlier_pipeline.fit(train_x.T)
    W_x = outlier_pipeline.transform(train_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, train_x.T)
    weight_x = cs_x.diagonal()
    idx = weight_x > train_threshold
    train_x = W_x[idx,:]
    train_y = train_y[idx]
    # outlier detection for test set
    W_x = outlier_pipeline.transform(test_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, test_x.T)
    weight_x = cs_x.diagonal()
    clip = weight_x > test_threshold
    if ~clip.all():
        print(f'{df_test[~clip]} is removed')
    test_x = W_x[clip,:]
    test_y = test_y[clip]
    df_test = df_test.iloc[clip]
    regressor_pipeline.fit(train_x, scalery.fit_transform(train_y.reshape((-1,1))).reshape(-1))
    pred = np.clip(scalery.inverse_transform(regressor_pipeline.predict(test_x).reshape((-1,1))), 0, 1)
    for i in range(pred.shape[0]):
        key = df_test.iloc[i][0] + str(df_test.iloc[i][2])
        pred_dict[key].append(pred[i])
        gt_dict[key].append(test_y[i])
isolation_window = ['A', 'B', 'C']
energy_level = [30 , 35, 40, 45, 50]
all_pred = []
all_gt = []
for (i, iw) in enumerate(isolation_window):
    for (j, el) in enumerate(energy_level):
        key = iw + str(el)
        rmse = mean_squared_error(pred_dict[key], gt_dict[key], squared=False)
        # f1 = f1_score(pred_dict[key], gt_dict[key])
        print(f'{iw}{el} | rmse: {"%.3f" % rmse}')
        all_pred = all_pred + pred_dict[key]
        all_gt = all_gt + gt_dict[key]

rmse = mean_squared_error(all_pred, all_gt, squared = False)
# f1 = f1_score(all_pred, all_gt)
print(f'all | rmse: {"%.3f" % rmse}')

NIST -> NIST
   0    1   2             3              4     5
3  A  CID  45  G2FS2(6)_MS5  G2FS2(6)_REDO  NIST is removed
    0    1   2             3              4     5
7   B  CID  40  G2FS2(6)_MS5  G2FS2(6)_REDO  NIST
10  C  CID  30  G2FS2(6)_MS5  G2FS2(6)_REDO  NIST
2   A  CID  40  G2FS2(6)_MS5  G2FS2(6)_REDO  NIST
12  C  CID  40  G2FS2(6)_MS5  G2FS2(6)_REDO  NIST is removed
    0    1   2             3              4     5
1   A  CID  35  G2FS2(6)_MS5  G2FS2(6)_REDO  NIST
14  C  CID  50  G2FS2(6)_MS5  G2FS2(6)_REDO  NIST is removed
A30 | rmse: 0.024
A35 | rmse: 0.046
A40 | rmse: 0.033
A45 | rmse: 0.022
A50 | rmse: 0.158
B30 | rmse: 0.063
B35 | rmse: 0.083
B40 | rmse: 0.023
B45 | rmse: 0.024
B50 | rmse: 0.069
C30 | rmse: 0.050
C35 | rmse: 0.120
C40 | rmse: 0.003
C45 | rmse: 0.037
C50 | rmse: 0.040
all | rmse: 0.068


In [28]:
# Intra-dataset prediction
print('all -> all')
from collections import defaultdict
from sklearn.model_selection import KFold
np_x, np_y,_, df_meta = load_datasets(datasets_SL+datasets_NIST)
idx = np.arange(np_y.shape[0])
np.random.shuffle(idx)
np_x = np_x[:, idx]
np_y = np_y[idx]
df_meta = df_meta.iloc[idx]
outlier_pipeline.fit(np_x.T)
pred_dict = defaultdict(lambda : [])
gt_dict = defaultdict(lambda : [])
k_fold = KFold(n_splits=5)
for trainidx, testidx in k_fold.split(np_x.T):
    train_x = np_x[:, trainidx]
    train_y = np_y[trainidx]
    test_x = np_x[:, testidx]
    test_y = np_y[testidx]
    df_test = df_meta.iloc[testidx]
    train_x = preprocessing_pipeline.fit_transform(train_x.T).T
    test_x = preprocessing_pipeline.transform(test_x.T).T
    # outlier detection for training set
    # outlier_pipeline.fit(train_x.T)
    W_x = outlier_pipeline.transform(train_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, train_x.T)
    weight_x = cs_x.diagonal()
    idx = weight_x > train_threshold
    train_x = W_x[idx,:]
    train_y = train_y[idx]
    # outlier detection for test set
    W_x = outlier_pipeline.transform(test_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, test_x.T)
    weight_x = cs_x.diagonal()
    clip = weight_x > test_threshold
    test_x = W_x[clip,:]
    test_y = test_y[clip]
    df_test = df_test.iloc[clip]
    regressor_pipeline.fit(train_x, scalery.fit_transform(train_y.reshape((-1,1))).reshape(-1))
    pred = np.clip(scalery.inverse_transform(regressor_pipeline.predict(test_x).reshape((-1,1))), 0, 1)
    for i in range(pred.shape[0]):
        key = df_test.iloc[i][0] + str(df_test.iloc[i][2])
        pred_dict[key].append(pred[i])
        gt_dict[key].append(test_y[i])
isolation_window = ['A', 'B', 'C']
energy_level = [30 , 35, 40, 45, 50]
all_pred = []
all_gt = []
for (i, iw) in enumerate(isolation_window):
    for (j, el) in enumerate(energy_level):
        key = iw + str(el)
        rmse = mean_squared_error(pred_dict[key], gt_dict[key], squared=False)
        # f1 = f1_score(pred_dict[key], gt_dict[key])
        print(f'{iw}{el} | rmse: {"%.3f" % rmse}')
        all_pred = all_pred + pred_dict[key]
        all_gt = all_gt + gt_dict[key]

rmse = mean_squared_error(all_pred, all_gt, squared = False)
# f1 = f1_score(all_pred, all_gt)
print(f'all | rmse: {"%.3f" % rmse}')

all -> all
A30 | rmse: 0.028
A35 | rmse: 0.043
A40 | rmse: 0.025
A45 | rmse: 0.037
A50 | rmse: 0.105
B30 | rmse: 0.050
B35 | rmse: 0.055
B40 | rmse: 0.037
B45 | rmse: 0.061
B50 | rmse: 0.036
C30 | rmse: 0.046
C35 | rmse: 0.079
C40 | rmse: 0.021
C45 | rmse: 0.023
C50 | rmse: 0.037
all | rmse: 0.051


In [21]:
# Predict released glycans
test_x, test_y, _,test_meta = load_datasets(datasets_Released)
train_x, train_y, _,train_meta = load_datasets(datasets_NIST+datasets_SL)
clip = np.where(test_meta[1] == 'CID')[0]
test_x = test_x[:,clip]
test_y = test_y[clip]
test_meta = test_meta.iloc[clip]
scalery = StandardScaler()
train_x = preprocessing_pipeline.fit_transform(train_x.T).T
test_x = preprocessing_pipeline.transform(test_x.T).T
# outlier detection for training set
outlier_pipeline.fit(np.concatenate((train_x.T, test_x.T)))
W_x = outlier_pipeline.transform(train_x.T)
est_x = outlier_pipeline.inverse_transform(W_x)
cs_x = cosine_similarity(est_x, train_x.T)
weight_x = cs_x.diagonal()
idx = weight_x > train_threshold
train_x = W_x[idx,:]
train_y = train_y[idx]
train_meta = train_meta.iloc[idx]
# outlier detection for test set
W_x = outlier_pipeline.transform(test_x.T)
est_x = outlier_pipeline.inverse_transform(W_x)
cs_x = cosine_similarity(est_x, test_x.T)
weight_x = cs_x.diagonal()
clip = weight_x > 0
test_x = W_x[clip,:]
test_y = test_y[clip]
test_meta = test_meta.iloc[clip]
model = regressor_pipeline.fit(train_x, scalery.fit_transform(train_y.reshape((-1,1))).reshape(-1))
print(f'training set samples:{train_x.shape[0]} test set samples:{test_x.shape[0]}')
pred_dict = defaultdict(lambda : [])
pred = np.clip(scalery.inverse_transform(model.predict(test_x).reshape((-1,1))), 0, 1)
for i in range(len(test_meta)):
    print(f'{"%.1f" % (100*pred[i])}({"%.1f" % (100*weight_x[i])})')
    if (i+1) % 15 == 0:
        print('\n')
    if weight_x[i] > test_threshold:
        # print(test_meta.iloc[i][4])
        pred_dict[test_meta.iloc[i][4]].append(pred[i])
for key in pred_dict.keys():
    print(f'{key}: {np.mean(pred_dict[key])} {np.std(pred_dict[key])}')

training set samples:168 test set samples:45
26.6(87.7)
17.1(80.7)
37.2(85.6)
14.9(90.5)
37.3(92.8)
52.6(48.2)
0.4(96.6)
7.6(94.2)
29.7(97.4)
10.7(88.3)
28.2(78.1)
20.3(90.6)
6.3(91.6)
3.1(96.4)
10.3(94.8)


0.0(96.9)
0.8(95.1)
0.8(92.8)
0.0(97.0)
0.0(95.7)
0.0(96.6)
0.0(94.8)
1.4(94.9)
0.0(96.4)
0.0(94.2)
0.0(98.9)
0.0(98.1)
2.2(96.2)
0.0(97.0)
0.0(96.1)


28.4(69.7)
11.8(88.3)
19.7(75.5)
1.5(93.0)
55.5(71.5)
41.2(76.6)
77.4(63.8)
38.8(72.1)
28.3(68.2)
41.6(59.6)
80.7(49.5)
33.4(64.0)
6.4(86.3)
53.1(52.6)
6.7(87.9)


ControlFetuin: 0.17043528080239712 0.11908539085888523
ControlTransferrin: 0.003445663724436439 0.006442280269116711
Alpha.2,3Neur.Fetuin: 0.06575510638895406 0.036375400569296605


In [ ]:
# List outliers
train_meta[train_meta['quality_score'] < 0.8]

,isolation_window,instrument,collision_energy,ratio,glycan_type,dataset,quality_score
46,A,CID,35,G2FS26_MS5,G2FS2(6),NIST,0.720166
47,A,CID,40,G2FS26_MS5,G2FS2(6),NIST,0.704256
49,A,CID,50,G2FS26_MS5,G2FS2(6),NIST,0.742414
50,B,CID,30,G2FS26_MS5,G2FS2(6),NIST,0.000000
54,B,CID,50,G2FS26_MS5,G2FS2(6),NIST,0.683916
57,C,CID,40,G2FS26_MS5,G2FS2(6),NIST,0.113300
58,C,CID,45,G2FS26_MS5,G2FS2(6),NIST,0.000000
59,C,CID,50,G2FS26_MS5,G2FS2(6),NIST,0.000000
61,A,CID,35,G2FS2(6)_MS5,G2FS2(6)_REDO,NIST,0.570558
62,A,CID,40,G2FS2(6)_MS5,G2FS2(6)_REDO,NIST,0.782714


In [30]:
# detect peaks in basis spectra
import scipy.signal as signal
test_x, test_y, _,test_meta = load_datasets(datasets_NIST_old, 'NMF_template')
train_x, train_y, mass_features,train_meta = load_datasets(datasets_SL)
scalery = StandardScaler()
train_x = preprocessing_pipeline.fit_transform(train_x.T).T
test_x = preprocessing_pipeline.transform(test_x.T).T
# outlier detection for training set
outlier_pipeline.fit(np.concatenate((train_x.T, test_x.T)))
W_x = outlier_pipeline.transform(train_x.T)
# detect W_x peaks
H = outlier_pipeline.steps[0][1].components_
H_sum = H[0,:]/np.max(H[0,:]) + H[1,:]/np.max(H[1,:])
peaks,_ = signal.find_peaks(H_sum, height=0.1, distance = 68)
mz_peaks = mass_features[peaks]
print(mz_peaks)
mass_tol = 33


[103.05746563 117.06026591 131.07766765 141.06506639 143.08006789
 145.09506939 173.11527141 175.13027291]


In [31]:
plt.plot(mass_features,H[1,:]/np.max(H[1,:]))

In [36]:
# mask each peak and see the influence on the prediction
for masked_peak in mz_peaks:
    print(f'masked peak: {masked_peak}')
    test_x, test_y, _,test_meta = load_datasets(datasets_NIST_old, 'NMF_template')
    train_x, train_y, _,train_meta = load_datasets(datasets_SL)
    mass_tol = 0.5
    mask = np.where((mass_features > masked_peak-mass_tol) & (mass_features < masked_peak+mass_tol))[0]
    train_x[mask,:] = 0
    test_x[mask,:] = 0
    scalery = StandardScaler()
    train_x = preprocessing_pipeline.fit_transform(train_x.T).T
    test_x = preprocessing_pipeline.transform(test_x.T).T
    # outlier detection for training set
    outlier_pipeline.fit(np.concatenate((train_x.T, test_x.T)))
    W_x = outlier_pipeline.transform(train_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, train_x.T)
    weight_x = cs_x.diagonal()
    idx = SL_NIST_train_idx
    train_x = W_x[idx,:]
    train_y = train_y[idx]
    train_meta = train_meta.iloc[idx]
    # outlier detection for test set
    W_x = outlier_pipeline.transform(test_x.T)
    est_x = outlier_pipeline.inverse_transform(W_x)
    cs_x = cosine_similarity(est_x, test_x.T)
    weight_x = cs_x.diagonal()
    clip = SL_NIST_test_idx
    test_x = W_x[clip,:]
    test_y = test_y[clip]
    test_meta = test_meta.iloc[clip]
    model = regressor_pipeline.fit(train_x, scalery.fit_transform(train_y.reshape((-1,1))).reshape(-1))
    print(f'training set samples:{train_x.shape[0]} test set samples:{test_x.shape[0]}')
    from collections import defaultdict
    pred_dict = defaultdict(lambda : [])
    gt_dict = defaultdict(lambda : [])
    pred = np.clip(scalery.inverse_transform(model.predict(test_x).reshape((-1,1))), 0, 1)
    for i in range(pred.shape[0]):
        key = test_meta.iloc[i][0] + str(test_meta.iloc[i][2])
        pred_dict[key].append(pred[i])
        gt_dict[key].append(test_y[i])
    isolation_window = ['A', 'B', 'C']
    energy_level = [30 , 35, 40, 45, 50]
    all_pred = []
    all_gt = []
    idx = 0
    diffs = []
    for (i, iw) in enumerate(isolation_window):
        for (j, el) in enumerate(energy_level):
            key = iw + str(el)
            # print(key)
            rmse = mean_squared_error(pred_dict[key], gt_dict[key], squared=False)
            s = ''
            if rmse > SL_NIST_rmse[idx]:
                s = '↑'
            else:
                s = '↓'
            relative = "%.2f" % (rmse/SL_NIST_rmse[idx] * 100) + '%'
            # f1 = f1_score(pred_dict[key], gt_dict[key])
            diffs.append(rmse-SL_NIST_rmse[idx])
            # print(f'{iw}{el} | rmse: {"%.3f" % rmse} | diff:{"%.3f" % (rmse-SL_NIST_rmse[idx])} {s}')
            print(f'{"%.3f" % (rmse-SL_NIST_rmse[idx])}({relative})', end='\n')
            all_pred = all_pred + pred_dict[key]
            all_gt = all_gt + gt_dict[key]

    rmse = mean_squared_error(all_pred, all_gt, squared = False)
    # f1 = f1_score(all_pred, all_gt)
    print(f'all | rmse: {"%.3f" % rmse} | avg diff:{"%.3f" % np.mean(diffs)}')

masked peak: 103.05746563092
training set samples:78 test set samples:73
-0.009(78.33%)
0.048(211.48%)
0.006(113.01%)
-0.010(77.51%)
0.034(179.84%)
0.011(126.56%)
0.082(290.91%)
-0.029(31.89%)
-0.043(0.00%)
0.004(110.25%)
0.044(201.89%)
0.069(262.38%)
-0.040(5.94%)
0.009(121.77%)
-0.022(48.78%)
all | rmse: 0.066 | avg diff:0.010
masked peak: 117.060265910948
training set samples:78 test set samples:73
0.196(558.16%)
0.301(804.27%)
0.141(429.38%)
0.114(366.21%)
0.188(541.07%)
0.081(288.90%)
0.422(1087.63%)
0.112(362.72%)
0.015(134.21%)
0.211(593.65%)
0.244(671.85%)
0.267(724.21%)
0.070(263.18%)
0.164(484.85%)
0.172(501.94%)
all | rmse: 0.250 | avg diff:0.180
masked peak: 131.077667651122
training set samples:78 test set samples:73
-0.043(0.00%)
-0.032(25.75%)
-0.010(77.17%)
-0.029(31.93%)
-0.026(38.10%)
-0.043(0.00%)
0.070(263.73%)
-0.043(0.00%)
-0.043(0.00%)
0.001(103.15%)
0.005(112.25%)
0.084(296.65%)
-0.043(0.00%)
-0.007(84.61%)
-0.043(0.00%)
all | rmse: 0.051 | avg diff:-0.013
maske